# SPQR

Tools using for creating and testing IDS/IPS rules

Developped by florian LOISY


In [ ]:
import os
import json
import datetime  # Importez le module datetime
from scripts.generate_traffic.spqrlib import (
    PcapGenerator,
)  # Assurez-vous que le nom du fichier est correct
from scripts.generate_path.folder import FolderGenerator
from scripts.process.process import SuricataExecution
import ipywidgets as widgets
from IPython.display import display
from os import path


## Définition des Données de la tâches à traiter

Veuillez compléter la cellule suivante avec les informations associées à la tâche en cours.

In [ ]:
'''
Cette cellule contient les variables à modifier en amont par l'analyste pour répondre au besoin à traiter.

Dans le cadre de la génération d'un fichier de capture réseau:
    - veuillez modifier les pattern à générer dans le fichier /spqr/config/config.json
    
Dans le cadre d'un test sur un fichier déjà existant:
    - copier le fichier dans le dossier /spqr/input
    - saisir le nom du fichier à la variable file_pcap_name de cette cellule.
'''

# Variables du nom de dossier

nom_dossier = "Regle_ANUBIS"
nom_bapteme = "ANUBIS"
tlp = "WHITE"  # saisir WHITE / GREEN / AMBER / RED
pap = "WHITE"  # saisir WHITE / GREEN / AMBER / RED

# Variables des options d'execution

test_with_pcap = True # saisir "True" ou "False" en fonction d'un test avec ou sans fichier de capture réseau
file_pcap_exist = False # saisir "True" ou "False" en fonction de la présence du fichier pcap pour le test
file_pcap_name = "file_YYYMMDDHHmmss.pcap" # indiquer le nom du fichier fourni que vous aurez placer dans le dossier "/input"





## Création des dossiers en OUTPUT

In [ ]:
# Variables du dossier d'output pour ce notebook

volume = os.path.abspath('./')  # indique le dossier de travail du projet

# Définition du dossier d'output pour ce notebook

fg = FolderGenerator()
pcap_path = fg.generate_folder(volume=volume, nom_dossier=nom_dossier)["pcap_path"]
result_path = fg.generate_folder(volume=volume, nom_dossier=nom_dossier)["result_path"]

# Initialisation de la variable process (NE PAS MODIFIER)

process = 0


## Définition du processus de test

In [ ]:
# Chargez la configuration à partir du fichier config.json

'''
Cette cellule a pour objectif de définir le déroulement des actions à venir en définissant le process à executer.
    process == 0 >> Une erreur est survenue
    process == 1 >> Le test ce fera sur un pcap fourni par l'analyste
    process == 2 >> Le test ce fera sur un pcap généré par un script python et l'outil SCAPY selon les variables définies par l'analyste dans le fichier config.json 
    process == 3 >> Le test ce fera sans traitement de pcap et fournira l'analyse syntaxique de la règle SURICATA par la sonde réseau SURICATA.

Il est actuellement possible de généré :
    des flux HTTP avec prise en compte du handshake TCP d'une requete client > server de la reponse du serveur et d'une fermeture de session
    des flux DNS avec une requete DNS client > server et une reponse du server > client
    des flux ICMP avec une requete type echo request et une reponse type echo-reply
'''

if test_with_pcap:
    if file_pcap_exist:
        file = pcap_path+"/"+file_pcap_name
        if not os.path.exists(file):
            copie_pcap = f"mv ./input/{file_pcap_name} {file}"
            os.system(copie_pcap)
            print(f"Le fichier {file_pcap_name} a été déplacé dans le dossier : {pcap_path}")
            
        else:
            pass

        print(f"Le test sera effectué sur le fichier {file}")
        output_pcap=file
        process = 1

    else:
        with open("config/config.json", "r") as config_file:
            config_data = json.load(config_file)
        process = 2
        
        # Créez des widgets pour chaque type de flux
        http_checkbox = widgets.Checkbox(description="HTTP", value=False)
        dns_checkbox = widgets.Checkbox(description="DNS", value=False)
        icmp_checkbox = widgets.Checkbox(description="ICMP", value=False) 
                    
else:
    process = 3
    output_pcap="-"
    print("Contrôle de syntaxe de la règle suricata")

## Génération d'un flux réseau





In [ ]:
# validation du process

'''
Cette cellule à pour objectif de générer un fichier de capture réseau ".pcap" si le process est défini à 2.
Dans un premier temps :
    Une box est ouverte et invite l'analyste à sélectionner le(s) protocole(s) réseau à générer
Dans un second temps :
    Un fichier .pcap est généré et enregistrer (par défaut) dans le dossier /spqr/output/<nom_dossier>/pcap/
'''


if process == 2:
    # Créez une instance de FlowGenerator avec un nom de fichier basé sur la date actuelle
    current_datetime = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    output_pcap = "output_" + current_datetime + ".pcap"

    # Créez un bouton pour générer les paquets
    generate_button = widgets.Button(description="Générer les paquets")
    generate_button.on_click(
        lambda x: PcapGenerator.create_packets(
            http_checkbox.value,
            dns_checkbox.value,
            icmp_checkbox.value,
            pcap_path,
            current_datetime,
        )
    )

    # Affichez les widgets et le bouton
    display(http_checkbox, dns_checkbox, icmp_checkbox, generate_button)

else:
    pass

## Lancement de SURICATA

In [ ]:
'''
Cette cellule sert à executer SURICATA dans une version dockerisée avec des options choisi selon le process à dérouler.
Les résultats sont disponibles (par défaut) dans le dossier /spqr/output/<nom_dossier>/result/<current_time>/
'''
#print(output_pcap)
suri=SuricataExecution()
suri.suricata_execution(process=process, result_path=result_path, pcap_path=pcap_path, nom_dossier=nom_dossier, file_pcap_name=file_pcap_name, output_pcap=output_pcap)
